In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.fluid.*
import kotlin.math.*

GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
// CDF Response Time Analysis Example 2
// Demonstrates class switching in a closed network with 3 classes

val model = Network("model")

// Create closed queueing network with Delay and Queue nodes
val node = arrayOfNulls<Node>(2)
node[0] = Delay(model, "Delay")
node[1] = Queue(model, "Queue2", SchedStrategy.PS)

// Three closed classes - only Class1 has initial population
val jobclass = arrayOfNulls<ClosedClass>(3)
jobclass[0] = ClosedClass(model, "Class1", 1, node[0]!!, 0)
jobclass[1] = ClosedClass(model, "Class2", 0, node[0]!!, 0)
jobclass[2] = ClosedClass(model, "Class3", 0, node[0]!!, 0)

// Class1 doesn't complete (stays in the system for class switching)
jobclass[0]!!.completes = false

In [ ]:
// Service processes at Delay node
node[0]!!.setService(jobclass[0]!!, Exp(1.0 / 1.0))  // rate = 1
node[0]!!.setService(jobclass[1]!!, Exp(1.0 / 1.0))  // rate = 1
node[0]!!.setService(jobclass[2]!!, Exp(1.0 / 1.0))  // rate = 1

// Service processes at Queue node
node[1]!!.setService(jobclass[0]!!, Exp(1.0 / 1.0))      // rate = 1
node[1]!!.setService(jobclass[1]!!, Erlang(1.0 / 2.0, 2)) // Erlang with rate=1/2, order=2
node[1]!!.setService(jobclass[2]!!, Exp(1.0 / 0.01))     // rate = 100

In [ ]:
// Complex routing matrix with class switching
val P = model.initRoutingMatrix()

// Class 1 to Class 1: Delay->Queue2
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)

// Class 1 to Class 2: Queue2->Delay (class switch)
P.set(jobclass[0], jobclass[1], node[1], node[0], 1.0)

// Class 2 to Class 1: Delay->Queue2 (class switch back)
P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)

// Class 2 to Class 2: Delay->Queue2
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)

// Class 3 circulates within itself
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)
P.set(jobclass[2], jobclass[2], node[1], node[0], 1.0)

model.link(P)

In [ ]:
// Solve with Fluid solver using state-dependent method
val solver = FLD(model, "method", "statedep", "iter_max", 100)
val avgRespT = solver.avgRespT
println("AvgRespT =")
avgRespT.print()

// Get CDF of response times
val FC = solver.cdfRespT

In [ ]:
// Calculate statistics from CDF
val numStations = model.numberOfStations
val numClasses = model.numberOfClasses
val avgRespTfromCDF = Array(numStations) { DoubleArray(numClasses) }
val powerMoment2R = Array(numStations) { DoubleArray(numClasses) }
val varianceR = Array(numStations) { DoubleArray(numClasses) }
val sqCoeffOfVariationRespTfromCDF = Array(numStations) { DoubleArray(numClasses) }

for (i in 0 until numStations) {
    for (c in 0 until numClasses) {
        if (FC != null && FC.size > i && FC[i] != null && FC[i].size > c && FC[i][c] != null && FC[i][c].numRows > 1) {
            val cdf = FC[i][c]
            var mean = 0.0
            var secondMoment = 0.0
            
            // Calculate moments from CDF using trapezoidal integration
            for (row in 1 until cdf.numRows) {
                val dt = cdf.get(row, 1) - cdf.get(row - 1, 1)
                val prob = cdf.get(row, 0)
                val time = cdf.get(row, 1)
                
                mean += dt * prob
                secondMoment += dt * prob * time
            }
            
            avgRespTfromCDF[i][c] = mean
            powerMoment2R[i][c] = secondMoment
            varianceR[i][c] = secondMoment - mean * mean
            if (mean > 0) {
                sqCoeffOfVariationRespTfromCDF[i][c] = varianceR[i][c] / (mean * mean)
            }
        }
    }
}

println("AvgRespTfromCDF =")
for (row in avgRespTfromCDF) {
    println(row.contentToString())
}
println("SqCoeffOfVariationRespTfromCDF =")
for (row in sqCoeffOfVariationRespTfromCDF) {
    println(row.contentToString())
}